<a target="_blank" href="https://colab.research.google.com/github/starccato/starccato_jax/blob/main/docs/example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Example 

## Train VAE

In [ ]:
%load_ext autoreload
%autoreload 2

! pip install starccato_jax -q

In [ ]:
import starccato_jax
from starccato_jax._version import __version__

print(f"starccato_jax version: {__version__}")

In [ ]:
import numpy as np

from starccato_jax.data import load_data
import matplotlib.pyplot as plt
import os

HERE = os.getcwd()
model_savedir = f"{HERE}/model_out"
os.makedirs(model_savedir, exist_ok=True)

train_data, val_data = load_data()

plt.figure(figsize=(3, 2.5))
for i in range(100):
    plt.plot(train_data[i], color="k", lw=0.3, alpha=0.2)
plt.savefig(f"{model_savedir}/training_data.png")

![](model_out/training_data.png)

In [ ]:
from starccato_jax.trainer import train_vae, Config

config = Config(latent_dim=16, epochs=300, cyclical_annealing_cycles=0)

train_vae(train_data, val_data, config, save_dir=model_savedir, plot_every=50)

|                     |          |
|---------------------|----------|
| **Loss**            |  ![loss] |  
| **Reconstructions** | ![reconstructions] | 

[loss]: model_out/loss.png
[reconstructions]: model_out/reconstructions.png

In [ ]:
from starccato_jax.io import load_model
from starccato_jax.model import generate
import jax

n = len(val_data)
zs = jax.random.uniform(jax.random.PRNGKey(0), (n, config.latent_dim))
model_data = load_model(model_savedir)

generated_signal = generate(model_data, z=zs)
for i in range(n):
    kwgs = dict(lw=0.1, alpha=0.1)
    plt.plot(generated_signal[i], color="tab:orange", **kwgs)
    plt.plot(val_data[i], color="k", **kwgs)

## MCMC on one validation dataset

In [ ]:
from starccato_jax.sampler import sample_latent_vars_given_data

mcmc_out = sample_latent_vars_given_data(
    train_data[54], model_savedir, rng_int=0, outdir=f"{HERE}/out_mcmc"
)

In [ ]:
import arviz as az

inf_data = az.from_numpyro(mcmc_out)
az.summary(inf_data, var_names=["z"])

|            | |
|------------|--|
| **Trace**  | ![trace_plot][trace_plot] |
| **Corner** | ![corner_plot][corner_plot] |
| **CI**     | ![ci_plot][ci_plot] |


[ci_plot]: out_mcmc/ci_plot.png
[corner_plot]: out_mcmc/corner_plot.png
[trace_plot]: out_mcmc/trace_plot.png